# 3.4 지도 학습 훈련 알아보기

지도학습에는 모델, 손실 함수, 훈련 데이터, 최적화 알고리즘이 필요합니다.  
지도학습의 훈련 데이터는 샘플과 타깃의 쌍입니다.  
모델은 샘플에 대한 예측을 계산하고, 손실 함수는 타깃과 비교하여 오차를 측정합니다.  
훈련 목표는 그레이디언트 기반의 최적화 알고리즘으로 모델의 파라미터를 조정하여 가능한 한 낮은 손실을 내는 것 입니다.  

한 클래스 포인트를 다른 클래스와 구별하는 것을 "결정 경계", 또는 "초평면" 이라고 합니다.

# 3.4.1 예제 데이터 만들기

머신러닝에서는 알고리즘을 이해하고자 할 때 일반적으로 설명하기 쉬운 성질의 합성 데이터를 만듭니다.

# 3.4.2 모델 선택

여기서 사용할 모델은 퍼셉트론입니다. 퍼셉트론은 어떤 크기의 입력도 다룰 수 있습니다.  
일반적인 모델 구축 과정에서 입력 크기는 문제와 데이터에 따라 결정됩니다.

퍼셉트론의 활성화 함수가 시그모이드 이므로 퍼센트론의 출력은 데이터 포인트(x)가 클래스 1일 확률입니다.

<img src = "3-2-1.jpg" width = "400px" height = "400px"></img>

# 3.4.3 확률을 클래스로 변환하기

<img src = "3-2-2.jpg" width = "400px" height = "400px"></img>

일반적으로 결정 경계는  0.5로 지정합니다. 하지만 실전에서 만족스러운 분류 정밀도를 얻으려면 (검증 데이터셋으로) 이 하이퍼파라미터를 수정해야 할 수도 있습니다.

# 3.4.4 옵티마이저 선택

모델이 예측을 만들고 손실 함수가 예측과 타깃 사이의 오차를 측정하면 옵티마이저가 이 오차 신호를 사용해 모델의 가중치를 업데이트 합니다.  
"학습률" 이라고 부르는 이 하이퍼파라미터는 오차 신호가 가중치 업데이트에 영향을 얼마나 미치는지 조절합니다.  
학습률이 크면 가중치가 크게 바뀌고, 수렴에 영향을 미칠 수 있습니다. 학습률이 너무 작으면 훈련 진행 속도가 심하게 느려질 수 있습니다.

In [17]:
# 코드 3-1 파이토치로 구현한 퍼셉트론
import torch
import torch.nn as nn

class Perceptron(nn.Module):
    """퍼셉트론은 하나의 선형 층입니다"""
    def _init_(self, input_dim):
        """
        매개변수:
            imput_dim (int): 입력 특성의 크기
        """
        super(Perceptron, self)._init_()
        self.fc1 = nn.Linear(input_dim, 1)
        
    def forward(self, x_in):
        """퍼셉트론의 정방향 계산
        매개변수:
            x_in (torch.Tensor): 입력 데이터 텐서
                x_in.shape는 (batch, num_features)입니다.
        반환값:
            결과 텐서. tensor.shape는 (batch,) 입니다.
        """
        return torch.sigmoid(self.fc1(x_in)).squeeze()

In [20]:
# 코드 3-10 Adam 옵티마이저 준비
import torch.nn as nn
import torch.optim as optim

input_dim = 2
lr = 0.001

perceptron = Perceptron(input_dim=input_dim)
bce_loss = nn.BCELoss()
optimizer = optim.Adam(params=percetron.parameters(), lr=lr)

TypeError: __init__() got an unexpected keyword argument 'input_dim'

# 3.4.6 모두 합치기: 그레이디언트 기반의 지도 학습

학습은 손실 계산에서 시작합니다. 즉, 모델의 예측이 타깃에서 얼마나 멀리 떨어져 있는지 측정합니다.  
손실의 그레이디언트는 모델 파라미터를 얼마나 많이 바꿔야 하는지 나타내는 신호가 됩니다.  
각 파라미터의 그레이디언트는 이 파라미터에 대한 손실값의 순간 변화율을 의미합니다.

그레이디언트 업데이트 알고리즘이 어떻게 동작할까요?
1. 모델(perceptron) 객체 안에 저장된 그레이디언트와 같은 부가 정보를 zero_grad()함수로 초기화 합니다.
2. 모델이 입력 데이터(x_data)에 대한 출력 (y_pred)을 계산합니다.
3. 모델 출력(y_pred)과 기대하는 타깃(y_target)을 비교해 손실을 계산합니다. 이것이 지도학습 훈련의 지도에 해당합니다.
4. 파이토치 손실 객체(criterion)에는 backward() 메서드가 있습니다. 이 메서드를 사용해 계산 그래프를 거슬러 손실을 반복해서 전파하고 각 파라미터에 대한 그레이디언트를 계산합니다.
5. 마지막으로 옵티마이저(opt)는 step() 함수로 파라미터에 그레이디언트를 업데이트 하는 방법을 지시합니다.

에포크 : 완전한 훈련 반복 한 번  
에포크 당 배치 개수가 데이터셋의 배치 개수와 같다면, 에포크가 데이터셋에 대한 완전한 반복 한 번이 됩니다.

In [15]:
# 코드 3-11 퍼셉트론과 이진 분류를 위한 지도 학습 훈련 반복
# 각 에포크는 전체 훈련 데이터를 사용합니다.
for eqoch_i in range(n_epochs):
    # 내부 반복은 데이터셋에 있는 배치에 대해 수행됩니다.
    for batch_i in range(n_batchs): 
        
        # 0단계: 데이터 가져오기
        x_data, t_target = get_toy_data(batch_size)
        
        # 1단계: 그레이디언트 초기화
        perceptron.zero_grad()
        
        # 2단계: 모델의 정방향 계산 수행하기
        y_pred = perceptron(x_data, apply_sigmoid=True)
        
        # 3단계: 최적하려는 손실 계산하기
        loss = bce_loss(y_pred, y_target)
        
        # 4단계: 손실 신호를 거꾸로 전파하기
        loss.backward()
         
        # 5단계: 옵티마이저로 업데이트하기
        optimizer.step()

NameError: name 'n_epochs' is not defined

# 3.5 부가적인 훈련 개념

# 3.5.1 모델 성능 올바르게 측정하기: 평가 지표

<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Ft1.daumcdn.net%2Fcfile%2Ftistory%2F99DC064C5BE056CE10" width = "400px" height = "400px"></img>

True Positive(TP) : 실제 True인 정답을 True 라고 예측 (정답)  
False Positive(FP) : 실제 False인 정답을 True 라고 예측 (오답)  
False Negative(FN) : 실제 True인 정답을 False라고 예측 (오답)  
True Negative(TN) : 실제 False인 정답을 False라고 예측 (정답)  

# 3.5.2 모델 성능 올바르게 측정하기: 데이터 분할

<img src = "https://velog.velcdn.com/images/pppanghyun/post/2a39921c-be87-4c53-ba92-3e5c4c06699d/image.png" width = "400px" height = "400px"></img>

K-fold 교차검증이란, "재표본을 추출"해서 모델을 학습하는 방법입니다.
1. 우선 train 데이터를 'k'개로 나눕니다.
2. 이후 한 번씩 돌아가면서(iteration) k개 만큼 나눈 데이터중 한 개를 test data로 사용하고 나머지는 train으로 사용합니다.  
이런경우 모든 데이터를 train에 사용할 수 있어 모델의 오버피팅을 방지하고 한정된 데이터셋으로도 안정적인 학습이 가능합니다.

# 3.5.3 훈련 중지 시점 파악하기

올바른 모델 성능을 측정하는 이유는 이 값을 사용해 훈련을 멈출 때를 결정하기 위해서 입니다.

가장 널리 사용하는 방법은 "조기종료" 입니다.  
조기 종료는 에포크마다 검증 데이터셋에 대한 성능을 기록하고 이 성능이 더는 좋아지지 않을 때를 감지합니다.

성능이 계속 좋아지지 않으면 훈련을 종료합니다. 훈련을 종료하기전에 기다리는 에포크 횟수를 "인내" 라고 합니다.  
일반적으로 모델이 어떤 데이터셋에서 개선되지 않는 지점을 "수렴"된 곳이라고 합니다.

# 3.5.4 최적의 하이퍼파라미터 찾기

하이퍼파라미터는 모델의 파라미터 개수와 값에 영향을 미치는 모든 모델 선정입니다.  
모델 훈련 방식을 결정하는 선택 옵션에는 손실함수, 옵티마이저, 옵티마이저의 학습률, 층 크기, 조기 종료하기전에 인내할 에포크 수, 다양한 규제 방법 등이 포함됩니다.

# 3.5.5 규제

### 1. L1 규제  
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2F3UYxP%2Fbtq92R7IFel%2Fftu7d7b2dAYwvqOfROFTHk%2Fimg.png" width = "400px" height = "400px"></img>  
앞부분은 실제값과 예측값을 대상으로 loss함수를 구한 값이고, 뒤의 부분이 L1 norm 이다. 람다값은 설정해야할 하이퍼파라미터이고, 1부터 n가지의 가중치의 절대값을 모두 더한 후 람다값을 곱한 값이 L1 norm이 된다.

Loss는 전체적으로 작아져야하기 때문에 람다값이 커지면 커질 수록 가중치 합은 줄어들 것 이다.(람다값이 커질수록 기울기가 작아짐)

L1 norm을 미분하면 가중치 부분은 사라지고 람다값에 w의 부호를 곱한 값이 된다.  
grad += lambda * np.sign(w)

### 2. L2 규제  
<img src = "https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fblog.kakaocdn.net%2Fdn%2FtfiG0%2Fbtq97ndWqpe%2FTXkMJpyH0f2x5F3DKIRpk1%2Fimg.png" width = "400px" height = "400px"></img>  
L1 규제의 경우 L1 norm을 미분하면서 가중치 부분이 부호만 남기고 사라지기 때문에 그레이디언트 값에 가중치를 반영할 수는 없다.  
l2 규제가 검증데이터의 손실을 잘 억제하면서 가중치를 완전히 0으로 만들지 않기 때문에 l1 보다는 l2를 많이 사용하는 추세이다.  

가중치를 제곱한 값의 합에 람다값을 곱한 값을 사용한다.  
l2 규제를 gradient값에 반영해 주기위해 미분하면 2lambda * w가 남는다. 따라서 해당 값을 구해진 그레디언트 값에 더하면 규제를 반영할 수 있다.  
grad += 2lambda * w